# Update EDA

base 모델에서 모든 Object 변수를 동일하게 더미변수화하여 처리하였으나, 교육 수준의 경우 단계별 순위가 있다는 것을 알 수 있음. 
- 가설 1. 교육 수준이 높으면 연간소득이 높다. 
- 가설 2. 교육 수준에 따라 신용도의 차이가 있다. 
- 가설 3. 연간 소득에 따라 신용도의 차이가 있다. 

In [1]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('card_dataset/train.csv')
test = pd.read_csv('card_dataset/test.csv')
submission= pd.read_csv('card_dataset/sample_submission.csv')

In [10]:
new_train = train.copy()
new_test = test.copy()

In [5]:
train['edu_type'].value_counts()

Secondary / secondary special    17995
Higher education                  7162
Incomplete higher                 1020
Lower secondary                    257
Academic degree                     23
Name: edu_type, dtype: int64

1) Academic degree  : 학위(5)    
2) Higher education : 고등학교 졸업(4)    
3) Incomplete higher : 고등학교 중퇴(3)    
4) Secondary / secondary special : 중학교(2)    
5) Lower secondary : 중학교 미만(1)    

In [15]:
edu_list = {'Academic degree':5,
           'Higher education':4,
           'Incomplete higher':3,
           'Secondary / secondary special':2,
           'Lower secondary':1}


new_train['edu_type'] = train['edu_type'].map(edu_list)

In [11]:
new_train['DAYS_BIRTH'] = np.round(np.abs(train['DAYS_BIRTH'])/365)  # 출생일 -> 나이로 변경 

In [17]:
new_train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,4,Married,Municipal apartment,38.0,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,2,Civil marriage,House / apartment,31.0,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,4,Married,House / apartment,52.0,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,2,Married,House / apartment,41.0,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,4,Married,House / apartment,41.0,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,2,Married,House / apartment,33.0,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,4,Separated,House / apartment,42.0,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,2,Civil marriage,With parents,28.0,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,3,Single / not married,House / apartment,28.0,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [18]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  int64  
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  float64
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [19]:
new_test['edu_type'] = test['edu_type'].map(edu_list)
new_test['DAYS_BIRTH'] = np.round(np.abs(test['DAYS_BIRTH'])/365)  # 출생일 -> 나이로 변경 

In [20]:
new_train[['work_phone','phone','email','FLAG_MOBIL']] = new_train[['work_phone','phone','email','FLAG_MOBIL']].astype('object')
new_test[['work_phone','phone','email','FLAG_MOBIL']] = new_test[['work_phone','phone','email','FLAG_MOBIL']].astype('object')

In [21]:
## 1. income_type = 'Pensioner' 일때 occyp_type = 'Retired' and DAYS_EMPLOYED 
new_train.loc[new_train['income_type'] == 'Pensioner', 'occyp_type'] = 'Retired'
new_train.loc[new_train['income_type'] == 'Pensioner', 'DAYS_EMPLOYED'] = 0 

## 2. occyp_type = NaN & income_type = 'State servant'인 경우 occyp_type ='State servant' 로 변경
new_train.loc[(new_train.occyp_type.isna())  & (new_train.income_type == 'State servant'), 'occyp_type'] = 'State servant'

## 3. occyp_type = NaN & income_type  in ('working', 'Commercial associate', 'Student) 일 경우 occyp_type ='Extra staff'로 변경 
new_train.loc[(new_train.occyp_type.isna())  & (new_train.income_type.isin(['Working','Commercial associate','Student'])), 'occyp_type'] = 'Extra staff'

## 1. income_type = 'Pensioner' 일때 occyp_type = 'Retired' and DAYS_EMPLOYED 
new_test.loc[new_test['income_type'] == 'Pensioner', 'occyp_type'] = 'Retired'
new_test.loc[new_test['income_type'] == 'Pensioner', 'DAYS_EMPLOYED'] = 0 

## 2. occyp_type = NaN & income_type = 'State servant'인 경우 occyp_type ='State servant' 로 변경
new_test.loc[(new_test.occyp_type.isna())  & (new_test.income_type == 'State servant'), 'occyp_type'] = 'State servant' 

## 3. occyp_type = NaN & income_type  in ('working', 'Commercial associate', 'Student) 일 경우 occyp_type ='Extra staff'로 변경 
new_test.loc[(new_test.occyp_type.isna())  & (new_test.income_type.isin(['Working','Commercial associate','Student'])), 'occyp_type'] = 'Extra staff' 


In [22]:
# 1. FLAG_MOBIL 변수 삭제
m_train = new_train.drop('FLAG_MOBIL', axis = 1)
m_test = new_test.drop('FLAG_MOBIL', axis =1)


# 2. credit 변수 분할 
target_credit = m_train['credit']
target_credit = target_credit.astype('int')

m_train_x = m_train.drop('credit', axis = 1)


# 3. 더미변수 
dum_train_x = pd.get_dummies(m_train_x, drop_first = True)
dum_test_x = pd.get_dummies(m_test, drop_first = True)

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [27]:
folds = StratifiedKFold(n_splits=5, 
                        shuffle=True, random_state=42)

In [43]:
outcomes = []
sub = np.zeros((dum_test_x.shape[0], 3))


for n_folds, (train_index, val_index) in enumerate(folds.split(dum_train_x, target_credit)):
    print(f'===================================={n_folds+1}============================================')
    
    X_train, X_val = dum_train_x.iloc[train_index], dum_train_x.iloc[val_index]
    y_train, y_val = target_credit.iloc[train_index], target_credit.iloc[val_index]
    
    evals = [(X_train, y_train),(X_val, y_val)]
    
    # Ligthgbm
    lgb = LGBMClassifier(n_estimators=20000,
                        random_state = 506,
                        learning_rate = 0.001,
                        max_depth = 16,
                        num_leaves = 2^15)
    
    lgb.fit(X_train, y_train, 
            eval_set= evals, 
            early_stopping_rounds=300,
           verbose=10000,
           eval_metric = 'multi_logloss')
        
    prediction = lgb.predict_proba(X_val)
    y_val_onehot = pd.get_dummies(y_val)
    
    logloss = log_loss(y_val_onehot, prediction)
    outcomes.append(logloss)
    
    sub+= lgb.predict_proba(dum_test_x)
    
    print(f'================================================================================\n\n')


====================================1============================================
Training until validation scores don't improve for 300 rounds
[10000]	training's multi_logloss: 0.734441	valid_1's multi_logloss: 0.7817
[20000]	training's multi_logloss: 0.685943	valid_1's multi_logloss: 0.771656
Did not meet early stopping. Best iteration is:
[20000]	training's multi_logloss: 0.685943	valid_1's multi_logloss: 0.771656


====================================2============================================
Training until validation scores don't improve for 300 rounds
[10000]	training's multi_logloss: 0.733819	valid_1's multi_logloss: 0.792371
[20000]	training's multi_logloss: 0.683559	valid_1's multi_logloss: 0.781951
Did not meet early stopping. Best iteration is:
[20000]	training's multi_logloss: 0.683559	valid_1's multi_logloss: 0.781951


====================================3============================================
Training until validation scores don't improve for 300 rounds
[10000]	

In [44]:
print(len(outcomes), np.mean(outcomes))

5 0.7749802025827129


In [45]:
submission.iloc[:,1:] = sub/5

In [46]:
submission.to_csv('submission/lgbm_chg_params.csv', index = False)